# ERA5 2m temperature variability

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, utils

plt.style.use("seaborn-v0_8-notebook")
plt.rcParams["figure.figsize"] = [15, 5]

## Define Parameters

In [ ]:
# Time period
year_start = 1940
year_stop = 2022

# Climatology periods
climatologies = (slice("1991", "2020"), slice("1981", "2010"), slice("1961", "1990"))

# Regions
regions = {
    "Global": {"lon_slice": slice(0, 360), "lat_slice": slice(90, -90)},
    "Tropics": {"lon_slice": slice(0, 360), "lat_slice": slice(30, -30)},
    "NH mid latitude": {"lon_slice": slice(0, 360), "lat_slice": slice(60, 30)},
    "SH mid latitude": {"lon_slice": slice(0, 360), "lat_slice": slice(-30, -60)},
    "Northern pole": {"lon_slice": slice(0, 360), "lat_slice": slice(90, 60)},
    "Southern pole": {"lon_slice": slice(0, 360), "lat_slice": slice(-60, -90)},
}

## Define request

In [ ]:
collection_id = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "time": "00:00",
    "format": "grib",
}

## Functions to cache

In [ ]:
def regionalised_spatial_weighted_mean(ds, lon_slice, lat_slice, **kwargs):
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    return diagnostics.spatial_weighted_mean(ds, **kwargs)

## Download and transform

In [ ]:
datasets = []
for region, transform_func_kwargs in regions.items():
    print(f"{region=}")
    ds = download.download_and_transform(
        collection_id,
        request,
        chunks={"year": 1},
        transform_func=regionalised_spatial_weighted_mean,
        transform_func_kwargs=transform_func_kwargs,
    )
    datasets.append(ds.expand_dims(region=[region]))

da = xr.concat(datasets, "region")["t2m"].reset_coords(drop=True).compute()

## Plot monthly climatologies

In [ ]:
dataarrays = []
for clima_slice in climatologies:
    da_clima = da.sel(forecast_reference_time=clima_slice)
    da_monthly = diagnostics.monthly_weighted_mean(da_clima, weights=False)
    dataarrays.append(
        da_monthly.expand_dims(period=[f"{clima_slice.start}-{clima_slice.stop}"])
    )
da_clima_monthly = xr.concat(dataarrays, "period")
da_clima_monthly.sel(region="Global").plot(hue="period")
_ = plt.grid()

## Plot monthly anomalies

In [ ]:
with xr.set_options(keep_attrs=True):
    da_anoma_monthly = da.groupby("forecast_reference_time.month") - da_clima_monthly
da_anoma_monthly.attrs["long_name"] += " anomaly"
da_anoma_monthly.sel(region="Global").plot(hue="period")
_ = plt.grid()

## Plot regional anomalies

In [ ]:
for regions_to_plot in (
    ["Tropics", "NH mid latitude", "SH mid latitude"],
    ["Northern pole", "Southern pole"],
):
    da_anoma_monthly.sel(region=regions_to_plot, period="1991-2020").plot(hue="region")
    plt.grid()
    plt.show()